# Duality

Each canonical linear programming problem is associated with a minimisation problem called the **dual** problem. Recall the canonical problem:

<img src="figures/lec02-canonical-linear-prog.png" width="500" />


The primal problem and the dual problem are given as follows:

<img src="figures/lec03-primal-and-dual-problem.png" width="400" />


## Converting between Primal and Dual

We can convert a primal problem into its dual by creating a table and performing a transpose operation.

<img src="figures/lec03-ex2-primal-problem.png" width="300" />

First, we put values in a table and perform transpose.

\begin{equation*}
\begin{bmatrix}
\begin{array}{ccccc|c}
x_1 & x_2 & b \\ \hline 
1 & 0 & 30 \\ 
0 & 1 & 20 \\ 
1 & 2 & 54 \\ \hline 
2 & 3 & 1 \\ 
\end{array}
\end{bmatrix}
\Longrightarrow^{Transpose}
\begin{bmatrix}
\begin{array}{ccccc|c}
y_1 & y_2 & y_3 & c \\ \hline 
1 & 0 & 1 & 2 \\ 
0 & 1 & 2 & 3 \\ \hline 
30 & 20 & 54 & 1 \\ 
\end{array}
\end{bmatrix}
\end{equation*}

From the resulting, we can read off the values for the dual problem.

<img src="figures/lec03-ex2-dual-problem.png" width="400" />


## Duality Theorem

The duality theorem states that the solution of one problem can be used as the solution its dual problem.


<img src="figures/lec03-duality-theorem.png" width="600" />


First, let us find the solution to primal problem:

In [1]:
import linprog as lp

A = [[1, 0],
     [0, 1],
     [1, 2]]
b = [30, 20, 54]
c = [2, 3]

res = lp.maximise(A, b, c)
lp.pretty_print(res)

<IPython.core.display.Math object>

We see that the optimal solution is $\bar{x} = (30, 12)$ and the maximal value $f(\bar{x}) = 96$

Next, we find the solution to the dual problem:

In [2]:
import linprog as lp

A = [[1, 0, 1],
     [0, 1, 2]]
b = [2, 3]
c = [30, 20, 54]

res = lp.minimise(A, b, c)
lp.pretty_print(res)

<IPython.core.display.Math object>

The optimal $\bar{y} = (1/2, 0, 3/2)$ and $g(\bar{y}) = 96$. This means that $f(\bar{x}) = g(\bar{y}) = 96$.

## Duality and the Simplex Method

There is another important relationship between duality and the simplex method, which is expressed in the Duality Theorem:


<img src="figures/lec03-duality-theorem-c.png" width="600" />


Let us use the simplex method on our running example.

In [3]:
from simplex import SimplexSolver 

A = [[1, 0],
     [0, 1],
     [1, 2]]
b = [30, 20, 54]
c = [2, 3]

res = SimplexSolver().run_simplex(A, b, c, prob='max', enable_msg=False, latex_path='latex/lec03-ex2-max.tex')

\begin{equation*}
\begin{bmatrix}
\begin{array}{ccccc|c}
x_1 & x_2 & s_1 & s_2 & s_3 & b \\ \hline 
0 & 0 & 1/2 & 1 & -1/2 & 8 \\ 
0 & 1 & -1/2 & 0 & 1/2 & 12 \\ 
1 & 0 & 1 & 0 & 0 & 30 \\ \hline 
0 & 0 & 1/2 & 0 & 3/2 & 96 \\ 
\end{array}
\end{bmatrix}
\begin{array}{c}\\ 
s_2 \\ 
x_2 \\ 
x_1 \\ 
\\ 
\end{array}\end{equation*}

When the entries in the bottom row are non-negative then we can read off optimal solution of the dual problem:


<img src="figures/lec03-ex2-duality-simplex.png" width="400" />


## Economic Interpretation of the Dual Problem

## Network Flow

## Matrix Games